In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import tensorflow_datasets as tfds

## Citation

```
@inproceedings{bossard14,
  title = {Food-101 -- Mining Discriminative Components with Random Forests},
  author = {Bossard, Lukas and Guillaumin, Matthieu and Van Gool, Luc},
  booktitle = {European Conference on Computer Vision},
  year = {2014}
}
```

#Data
TensorFlow food 101 dataset.

https://www.tensorflow.org/datasets/catalog/food101

label of Hotdog is 55

In [ ]:
ds, ds_info = tfds.load('food101', shuffle_files=True, as_supervised=True, with_info=True)

<details>
<summary>code explanation</summary>

```tfds.load('food101', shuffle_files=True, as_supervised=True, with_info=True):```

This function is used to load the 'food101' dataset from TensorFlow Datasets (tfds).

Parameters:
'food101': Specifies the name of the dataset to be loaded.

shuffle_files=True: Specifies that the dataset files should be shuffled.

as_supervised=True: Loads the dataset in a supervised format, where inputs and labels are returned as tuples.

with_info=True: Retrieves additional information about the dataset, such as the number of classes, size, etc.

Returns:

ds: The loaded dataset.

ds_info: Additional information about the dataset.
</details>

In [ ]:
train_ds, valid_ds = ds["train"], ds["validation"]

In [ ]:
fig = tfds.show_examples(train_ds, ds_info)

In [ ]:
MAX_SIDE_LEN = 128
HOT_DOG_CLASS = 55
train_ds = train_ds.map(
    lambda image, label: (tf.cast(tf.image.resize(image, [MAX_SIDE_LEN, MAX_SIDE_LEN]), dtype=tf.int32), tf.cast(label == HOT_DOG_CLASS, dtype=tf.int32))
)
valid_ds = valid_ds.map(
    lambda image, label: (tf.cast(tf.image.resize(image, [MAX_SIDE_LEN, MAX_SIDE_LEN]), dtype=tf.int32), tf.cast(label == HOT_DOG_CLASS, dtype=tf.int32))
)

<details>
<summary>code explanation</summary>

```tf.data.Dataset.map(lambda image, label: (..., ...)):```

This function is used to apply a mapping function to each element of a dataset.

In this code, it is used twice to preprocess the training and validation datasets:

It resizes the images to a maximum side length of 128 pixels using
```tf.image.resize.```

It converts the labels of hotdogs (class 55) to 1 and other classes to 0 using ```tf.cast.```
</details>

In [ ]:
fig = tfds.show_examples(train_ds, ds_info)

In [ ]:
train_hd_size, valid_hd_size = 750, 250
train_hotdogs = train_ds.filter(lambda image, label: label == 1).repeat(3)
train_nothotdogs = train_ds.filter(lambda image, label: label == 0)

In [ ]:
valid_hotdogs = train_ds.filter(lambda image, label: label == 1).repeat(3)
valid_nothotdogs = train_ds.filter(lambda image, label: label == 0)

<details>
<summary>code explanation</summary>

```tf.data.Dataset.filter(lambda image, label: ...):```

This function is used to filter the elements of a dataset based on a given condition.

In this code, it is used to create separate datasets for hotdogs and not hotdogs in the training and validation sets.
</details>

In [ ]:
batch_size = 16
train_ds = tf.data.Dataset.sample_from_datasets([train_hotdogs, train_nothotdogs],[0.5,0.5], stop_on_empty_dataset=True)
train_ds = train_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

valid_ds = tf.data.Dataset.sample_from_datasets([valid_hotdogs, valid_nothotdogs],[0.5,0.5], stop_on_empty_dataset=True)
valid_ds = valid_ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

<details>
<summary>code explanation</summary>
```tf.data.Dataset.sample_from_datasets([dataset1, dataset2], [weights1, weights2], stop_on_empty_dataset=True):```

This function is used to sample elements from multiple datasets based on the provided weights.

In this code, it is used to create balanced training and validation datasets by sampling equal proportions of hotdog and not hotdog samples.

```tf.data.Dataset.cache():```

This function is used to cache the elements of a dataset in memory or on disk.

In this code, it is used to cache the training and validation datasets, which can improve training performance by avoiding repeated data loading and preprocessing.

```tf.data.Dataset.batch(batch_size):```

This function is used to combine consecutive elements of a dataset into batches.
In this code, it is used to create batches of a specified size for training and validation datasets.

```tf.data.Dataset.prefetch(tf.data.AUTOTUNE):```

This function is used to optimize data loading by prefetching elements from a dataset while the model is training on the current batch.
In this code, it is used to prefetch elements from the training and validation datasets to improve training performance.
</details>

In [ ]:
for image_batch, label_batch in train_ds.take(1):
  print(image_batch)
  print(label_batch)

#Neural network

In [ ]:
data_augumentaion = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2)
])

<details>
<summary>code explanation</summary>
```tf.keras.Sequential():```

This function is used to create a sequential model in Keras.

In this code, it is used to define the neural network model architecture.

```tf.keras.layers.RandomFlip('horizontal'):```

This function is used to randomly flip the images horizontally during training for data augmentation.

It helps the model generalize better by providing additional variations of the training data.

```tf.keras.layers.RandomRotation(0.2):```

This function is used to randomly rotate the images by a maximum angle of 0.2 radians during training for data augmentation.

It introduces further variations in the training data, making the model more robust to rotation.

<details>

In [ ]:
for i,_ in ds["train"].take(1):
  image = i

In [ ]:
plt.imshow(image)

In [ ]:
image = tf.cast(tf.expand_dims(image,0), tf.float32)
image /= 255.0

In [ ]:
plt.figure(figsize=(10,10))
for i in range(9):
  augumented_image = data_augumentaion(image)
  ax = plt.subplot(3,3, i + 1)
  plt.imshow(augumented_image[0])
  plt.axis("off")

In [ ]:
random.seed(0)
model = models.Sequential()
model.add(layers.Rescaling(1./255))
model.add(data_augumentaion)
model.add(layers.Conv2D(64, (3,3), activation="relu", input_shape=[MAX_SIDE_LEN, MAX_SIDE_LEN, 3]))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3,3), activation="relu", kernel_regularizer = tf.keras.regularizers.l2(l = 0.1)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(32, (3,3), activation="relu", kernel_regularizer = tf.keras.regularizers.l2(l = 0.1)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1))

<details>
<summary>code explanation</summary>
```tf.keras.layers.Rescaling(1./255):```

This layer is used to normalize the pixel values of the input images between 0 and 1 by dividing them by 255.

Normalizing the input data helps in faster convergence during training.

```tf.keras.layers.Conv2D(filters, kernel_size, activation, input_shape):```

This layer is used to add a 2D convolutional layer to the model.

It performs convolutional operations on the input data to extract features.
Parameters:

filters: The number of filters in the convolutional layer.

kernel_size: The size of the filters.

activation: The activation function to be applied.

input_shape: The shape of the input data.

```tf.keras.layers.MaxPooling2D(pool_size):```

This layer is used to add a max pooling layer to the model.

It downsamples the input data by taking the maximum value within each pooling window.

Parameter:

pool_size: The size of the pooling window.

```tf.keras.layers.Dropout(rate):```

This layer is used to apply dropout regularization to the model.

It randomly sets a fraction of input units to 0 during training, which helps in reducing overfitting.

Parameter:

rate: The fraction of input units to drop.

```tf.keras.layers.Flatten():```

This layer is used to flatten the multi-dimensional input into a 1D vector.

It prepares the data for the fully connected layers.

```tf.keras.layers.Dense(units, activation):```

This layer is used to add a fully connected layer to the model.

It connects every input unit to every output unit.

Parameters:

units: The number of neurons in the layer.

activation: The activation function to be applied.
</details>

In [ ]:
lr = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

<details>
<summary>code explanation</summary>

```tf.keras.optimizers.Adam(learning_rate):```

This optimizer is used to update the model's parameters during training using the Adam optimization algorithm.

It adapts the learning rate based on the moving averages of the gradient.

Parameter:

learning_rate: The learning rate for the optimizer.

```tf.keras.losses.BinaryCrossentropy(from_logits=True):```

This loss function is used for binary classification problems.

It computes the cross-entropy loss between the true labels and predicted logits.

Parameter:

from_logits=True: Specifies that the model's output is not normalized with a sigmoid activation.
</details>

In [ ]:
epochs = 15
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=epochs,
    verbose=1
)

<details>
<summary>code explanation</summary>

```model.fit(train_ds, validation_data, epochs, verbose):```

This function is used to train the model.

It fits the model to the training data and evaluates it on the validation data for the specified number of epochs.

Parameters:

train_ds: The training dataset.

validation_data: The validation dataset.

epochs: The number of epochs for training.

verbose: Controls the verbosity mode (0 = silent, 1 = progress bar, 2 = one line per epoch).

```plt.imshow(image)``` and ```plt.show():```

These functions are used to display an image using matplotlib.

They are used to visualize the images from the dataset and the predicted labels.
</details>

In [ ]:
plt.figure(figsize=(10,10))
for image_batch, label_batch in valid_ds.take(1):
  images = image_batch
  labels = label_batch

In [ ]:
for i in range(9):
  ax = plt.subplot(3,3, i+1)
  plt.imshow(images[i])
  plt.axis("off")

In [ ]:
labels[:9]